In [1]:
import pandas as pd
import requests
from config import api_key
import json

In [2]:
rep_candidate=pd.read_csv("rep_candidate.csv")
new_gop_finance=pd.read_csv("new_gop_finance.csv")

In [4]:
rep_candidate=rep_candidate.rename(columns={
    "state":"candidate_state"
})
rep_finance=new_gop_finance.rename(columns={
    "state":"donor_state"
})



In [5]:
rep_data=pd.DataFrame.merge(rep_candidate, rep_finance, on="candidate_id")

In [9]:
rep_data=rep_data.rename(columns={
    "state":"donor_state"
})
rep_data

,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,candidate_status,donor_state,donation_count,total_raised
0,"ACKISON, MELISSA",S8OH00094,OH,REP,Challenger,Senate,[2018],[2018],C,AZ,1,500.00
1,"ACKISON, MELISSA",S8OH00094,OH,REP,Challenger,Senate,[2018],[2018],C,OH,88,57658.02
2,"ACKISON, MELISSA",S8OH00094,OH,REP,Challenger,Senate,[2018],[2018],C,OT,82,13050.30
3,"ANDERSON, BOB",S8MN00594,MN,REP,Challenger,Senate,[2018],[2018],C,OT,1,250.00
4,"ANDERSON, CONSUELO E",S8HI00194,HI,REP,Challenger,Senate,[2018],[2018],N,CA,1,1000.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1954,"NEWBERGER, JAMES WALLACE",S8MN00552,MN,REP,Challenger,Senate,"[2018, 2020]",[2018],P,FL,1,250.00
1955,"NEWBERGER, JAMES WALLACE",S8MN00552,MN,REP,Challenger,Senate,"[2018, 2020]",[2018],P,IL,1,250.00
1956,"NEWBERGER, JAMES WALLACE",S8MN00552,MN,REP,Challenger,Senate,"[2018, 2020]",[2018],P,KY,1,100.00
1957,"NEWBERGER, JAMES WALLACE",S8MN00552,MN,REP,Challenger,Senate,"[2018, 2020]",[2018],P,MN,278,127402.07


In [10]:
# create data frame with totals fundraised by every candidate in every state they fundraised

rep_data=rep_data.filter(["name","candidate_id","candidate_state","party","incumbent_challenge_full","office_full","cycles","election_years",
                        "candidate_status","donor_state","donation_count","total_raised"])
# rep_data.to_csv("rep_delete.csv")

In [11]:
# if candidate_state = donor_state, in_state=True, if not, then False

in_state=(rep_data.candidate_state==rep_data.donor_state)
in_state=in_state.to_frame()
in_state=in_state.rename(columns={
    0: "in_state"
})
in_state

,in_state
0,False
1,True
2,False
3,False
4,False
...,...
1954,False
1955,False
1956,False
1957,True


In [12]:
new_rep_data=pd.DataFrame.merge(rep_data,in_state, left_index=True, right_index=True)

new_rep_data.set_index("candidate_id")
new_rep_data.to_csv("rep_totals_by_state.csv")

In [22]:
new_rep_data["candidate_id"].nunique()
rep_aggregate=new_rep_data.groupby(["candidate_id", "in_state"]).agg({"donation_count":["sum"], "total_raised":["sum"]})
rep_aggregate


rep_aggregate.columns = rep_aggregate.columns.droplevel(1)

rep_aggregate=rep_aggregate.reset_index()

in_state=rep_aggregate.loc[rep_aggregate["in_state"]==True]
in_state
out_of_state=rep_aggregate.loc[rep_aggregate["in_state"]==False]
out_of_state

state_agg=pd.DataFrame.merge(in_state, out_of_state, on="candidate_id", suffixes=("_in_state","_out_of_state"), how="outer")
# state_agg.to_csv("rep_finance_data.csv")

rep_candidates=pd.read_csv("rep_candidate.csv")
final_rep_data=pd.DataFrame.merge(rep_candidates, state_agg, on="candidate_id")
final_rep_data.head()

,Unnamed: 0,name,candidate_id,state,party,incumbent_challenge_full,office_full,cycles,election_years,party.1,candidate_status,in_state_in_state,donation_count_in_state,total_raised_in_state,in_state_out_of_state,donation_count_out_of_state,total_raised_out_of_state
0,0,"ACKISON, MELISSA",S8OH00094,OH,REP,Challenger,Senate,[2018],[2018],REP,C,True,88.0,57658.02,False,83.0,13550.30
1,2,"ANDERSON, BOB",S8MN00594,MN,REP,Challenger,Senate,[2018],[2018],REP,C,NaN,NaN,NaN,False,1.0,250.00
2,3,"ANDERSON, CONSUELO E",S8HI00194,HI,REP,Challenger,Senate,[2018],[2018],REP,N,NaN,NaN,NaN,False,3.0,13000.00
3,4,"ARLETT, ROB",S8DE00145,DE,REP,Challenger,Senate,"[2018, 2020]",[2018],REP,P,True,204.0,182484.76,False,21.0,14009.92
4,5,"ARPAIO, JOE",S8AZ00247,AZ,REP,Open seat,Senate,"[2018, 2020]",[2018],REP,P,True,141.0,37652.00,False,1698.0,381971.76


In [23]:
final_rep_data=final_rep_data.filter(["name","candidate_id","state", "party","incumbent_challenge_full","cycles","donation_count_in_state","total_raised_in_state","donation_count_out_of_state","total_raised_out_of_state"])

In [25]:
final_rep_data.to_csv("final_rep_data.csv")